In [1]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from torch.nn import BCEWithLogitsLoss

/home/danbee/anaconda3/envs/danbee/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os

print(os.getcwd())

/home/danbee/Desktop/Ansim_Keypad/MODEL/텍스트_윤리검증_모델


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [4]:
# Load the model and tokenizer
model_name = 'BM-K/KoSimCSE-bert-multitask'
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the dataset
df = pd.read_csv("../../DATA/텍스트_윤리검증_데이터/emotion_labeld_text.csv")
# df = df[:10000]


/home/danbee/anaconda3/envs/danbee/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
labels = df.drop(['conversation_idx', 'sentence_idx', 'origin_text', 'speaker', 'intensity'], axis=1) # intensity 추가됨
print(labels.columns)

Index(['is_immoral', '불평/불만', '환영/호의', '감동/감탄', '지긋지긋', '고마움', '슬픔', '화남/분노',
       '존경', '기대감', '우쭐댐/무시함', '안타까움/실망', '비장함', '의심/불신', '뿌듯함', '편안/쾌적',
       '신기함/관심', '아껴주는', '부끄러움', '공포/무서움', '절망', '한심함', '역겨움/징그러움', '짜증',
       '어이없음', '없음', '패배/자기혐오', '귀찮음', '힘듦/지침', '즐거움/신남', '깨달음', '죄책감',
       '증오/혐오', '흐뭇함(귀여움/예쁨)', '당황/난처', '경악', '부담/안_내킴', '서러움', '재미없음',
       '불쌍함/연민', '놀람', '행복', '불안/걱정', '기쁨', '안심/신뢰', 'DISCRIMINATION', 'HATE',
       'CENSURE', 'VIOLENCE', 'CRIME', 'SEXUAL', 'ABUSE'],
      dtype='object')


In [6]:
# 모든 항목에 대해 0.5 이상이면 1, 0.5 미만이면 0으로 바꾸기
df = df.apply(lambda x: x if x.name in ['conversation_idx', 'sentence_idx', 'origin_text', 'speaker', 'intensity'] else x.apply(lambda y: 1 if y >= 0.5 else 0))

In [7]:
# reset memory
torch.cuda.empty_cache()

In [8]:
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import torch
import pandas as pd
from torch.nn import BCEWithLogitsLoss
from tqdm import tqdm

# Determine the number of labels
num_labels = len(df.columns) - 5  # Subtracting non-label columns: 'conversation_idx', 'sentence_idx', 'origin_text', 'speaker'

# Load the model with the correct number of labels
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# Train-test split
train, val = train_test_split(df, test_size=0.001, random_state=42)

# Custom Dataset Class
class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer):
        self.data = dataframe
        self.tokenizer = tokenizer
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        text = self.data.iloc[idx]['origin_text']
        labels = self.data.drop(['conversation_idx', 'sentence_idx', 'origin_text', 'speaker', 'intensity'], axis=1).iloc[idx].values.astype('float32') # intensity 추가됨
        inputs = self.tokenizer(text, padding="max_length", truncation=True, return_tensors="pt")
        inputs = {k: v.squeeze() for k, v in inputs.items()}
        inputs['labels'] = torch.tensor(labels)
        return inputs

# Create Dataset instances
train_dataset = TextDataset(train, tokenizer)
val_dataset = TextDataset(val, tokenizer)


class CustomTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.writer = SummaryWriter(log_dir='./logs')
        self.progress_bar = tqdm(total=self.args.max_steps, desc="Training Progress")

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = BCEWithLogitsLoss()
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

    def training_step(self, model, inputs):
        loss = super().training_step(model, inputs)
        self.writer.add_scalar('Loss/train', loss.item(), self.state.global_step)
        self.progress_bar.update(1)
        return loss

    def evaluate(self, eval_dataset=None, ignore_keys=None, metric_key_prefix: str = "eval"):
        metrics = super().evaluate(eval_dataset, ignore_keys, metric_key_prefix)
        self.writer.add_scalar('Loss/validation', metrics['eval_loss'], self.state.global_step)
        
        # Log other metrics if available
        for key, value in metrics.items():
            if key.startswith(metric_key_prefix):
                self.writer.add_scalar(f'{metric_key_prefix}/{key}', value, self.state.global_step)
        
        return metrics

    def log_metrics(self, split, metrics):
        # Log metrics to TensorBoard
        for key, value in metrics.items():
            self.writer.add_scalar(f'{split}/{key}', value, self.state.global_step)
        super().log_metrics(split, metrics)

    def train(self, *args, **kwargs):
        self.progress_bar.reset()
        super().train(*args, **kwargs)
        self.progress_bar.close()
        self.writer.close()  # Ensure the writer is closed after training

# Set up training arguments and the Trainer
training_args = TrainingArguments(
    output_dir='./results',          
    num_train_epochs=3,              
    per_device_train_batch_size=32,  
    per_device_eval_batch_size=32,   
    warmup_steps=500,                
    weight_decay=0.01,               
    logging_dir='./logs',            
    save_steps=3000,                 
    save_total_limit=2,              
    report_to="tensorboard",         
    eval_steps=200,
    # logging_steps=100,  # Ensure logging occurs frequently
    evaluation_strategy="steps",  # Evaluate at every eval_steps
)

trainer = CustomTrainer(
    model=model,                         
    args=training_args,                  
    train_dataset=train_dataset,         
    eval_dataset=val_dataset,            
)

# Start training
trainer.train(resume_from_checkpoint=True)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at BM-K/KoSimCSE-bert-multitask and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/danbee/anaconda3/envs/danbee/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Training Progress: 200it [06:09,  1.85s/it]











                                           
                                                   

Training Progress: 200it [06:14,  1.85s/it]    



{'eval_loss': 0.3077302873134613, 'eval_runtime': 4.6227, 'eval_samples_per_second': 78.742, 'eval_steps_per_second': 2.596, 'epoch': 0.02}


Training Progress: 400it [12:22,  1.84s/it]











                                           
                                                   

Training Progress: 400it [12:27,  1.84s/it]    



{'eval_loss': 0.240280881524086, 'eval_runtime': 4.6583, 'eval_samples_per_second': 78.141, 'eval_steps_per_second': 2.576, 'epoch': 0.04}


                                           
Training Progress: 500it [15:31,  1.83s/it]           

...

{'eval_loss': 0.13747155666351318, 'eval_runtime': 7.104, 'eval_samples_per_second': 51.239, 'eval_steps_per_second': 1.689, 'epoch': 0.71}


Training Progress: 8200it [4:06:45,  1.74s/it]











                                              

                                         
Training Progress: 8200it [4:06:52,  1.74s/it]           



{'eval_loss': 0.13710305094718933, 'eval_runtime': 6.7458, 'eval_samples_per_second': 53.959, 'eval_steps_per_second': 1.779, 'epoch': 0.72}


Training Progress: 8352it [4:11:18,  1.86s/it]

KeyboardInterrupt: 

In [3]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the saved model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained('./results/checkpoint-5000')
tokenizer = AutoTokenizer.from_pretrained('BM-K/KoSimCSE-bert-multitask')

# Move model to device
model.to(device)

# Define the test dataset
class TestDataset(Dataset):
    def __init__(self, dataframe, tokenizer):
        self.data = dataframe
        self.tokenizer = tokenizer
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        text = self.data.iloc[idx]['origin_text']
        inputs = self.tokenizer(text, padding="max_length", truncation=True, return_tensors="pt")
        inputs = {k: v.squeeze() for k, v in inputs.items()}
        return inputs
df = pd.read_csv("../../DATA/텍스트_윤리검증_데이터/emotion_labeld_text.csv")
# Load the test data
test_df = df[:100]
test_dataset = TestDataset(test_df, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=16)

# Evaluate the model
model.eval()
predictions = []

with torch.no_grad():
    for batch in test_loader:
        inputs = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**inputs)
        logits = outputs.logits
        preds = torch.sigmoid(logits).cpu().numpy()  # Move tensor to CPU before converting to NumPy
        predictions.extend(preds)

label_columns = ['is_immoral', '불평/불만', '환영/호의', '감동/감탄', '지긋지긋', '고마움',
       '슬픔', '화남/분노', '존경', '기대감', '우쭐댐/무시함', '안타까움/실망', '비장함', '의심/불신', '뿌듯함',
       '편안/쾌적', '신기함/관심', '아껴주는', '부끄러움', '공포/무서움', '절망', '한심함', '역겨움/징그러움',
       '짜증', '어이없음', '없음', '패배/자기혐오', '귀찮음', '힘듦/지침', '즐거움/신남', '깨달음', '죄책감',
       '증오/혐오', '흐뭇함(귀여움/예쁨)', '당황/난처', '경악', '부담/안_내킴', '서러움', '재미없음',
       '불쌍함/연민', '놀람', '행복', '불안/걱정', '기쁨', '안심/신뢰', 'DISCRIMINATION', 'HATE',
       'CENSURE', 'VIOLENCE', 'CRIME', 'SEXUAL', 'ABUSE']

# Convert predictions to a DataFrame
predictions_df = pd.DataFrame(predictions, columns=label_columns)

/home/danbee/anaconda3/envs/danbee/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
# Assuming the model is already moved to the GPU

examples = ['너', '너 정말', '너 정말 개', '너 정말 개새끼', "너 정말 개새끼구나"]
predictions = pd.DataFrame(columns=label_columns)

for example in examples:
    inputs = tokenizer(example)
    inputs = {k: torch.tensor(v).unsqueeze(0).to(device) for k, v in inputs.items()}  # Move inputs to GPU
    outputs = model(**inputs)
    preds = torch.sigmoid(outputs.logits).detach().cpu().numpy()  # Detach tensor before converting to NumPy
    
    # Create a DataFrame for the current prediction
    current_prediction = pd.DataFrame(preds, columns=label_columns)  # Exclude the "queue" column
    current_prediction["queue"] = example  # Add the "queue" column
    
    # Concatenate the current prediction to the predictions DataFrame
    predictions = pd.concat([predictions, current_prediction], axis=0, ignore_index=True)

# Reorder columns to have "queue" as the first column
predictions = predictions[["queue"] + label_columns[:-1]]

predictions

/tmp/ipykernel_160967/1672186519.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictions = pd.concat([predictions, current_prediction], axis=0, ignore_index=True)


,queue,is_immoral,intensity,불평/불만,환영/호의,감동/감탄,지긋지긋,고마움,슬픔,화남/분노,...,행복,불안/걱정,기쁨,안심/신뢰,DISCRIMINATION,HATE,CENSURE,VIOLENCE,CRIME,SEXUAL
0,너,0.179760,0.038211,0.350679,0.096618,0.046731,0.314059,0.015149,0.033535,0.260198,...,0.024146,0.264991,0.044902,0.070248,0.030641,0.026364,0.165029,0.004865,0.007795,0.021417
1,너 정말,0.242867,0.089174,0.498137,0.036678,0.135842,0.157985,0.010959,0.156921,0.577039,...,0.015450,0.311582,0.030134,0.024001,0.022595,0.036645,0.202873,0.005243,0.006627,0.012880
2,너 정말 개,0.600446,0.396136,0.397350,0.042395,0.135958,0.089429,0.007664,0.088975,0.597812,...,0.013392,0.288172,0.028436,0.016959,0.053053,0.100659,0.388168,0.017173,0.013358,0.029789
3,너 정말 개새끼,0.663998,0.999970,0.367194,0.073576,0.155910,0.065485,0.012745,0.085823,0.587879,...,0.025691,0.189923,0.051150,0.013936,0.043211,0.112162,0.489692,0.022525,0.014018,0.031690
4,너 정말 개새끼구나,0.630266,0.824170,0.312282,0.074153,0.317904,0.063481,0.012726,0.085510,0.517674,...,0.028518,0.190189,0.072226,0.021714,0.054911,0.101805,0.482196,0.015326,0.013765,0.043446


In [46]:
# Assuming the model is already moved to the GPU

examples = ["너희", "너희 집", "너희 집 개", "너희 집 개 귀엽다" ]
predictions = pd.DataFrame(columns=label_columns)

for example in examples:
    inputs = tokenizer(example)
    inputs = {k: torch.tensor(v).unsqueeze(0).to(device) for k, v in inputs.items()}  # Move inputs to GPU
    outputs = model(**inputs)
    preds = torch.sigmoid(outputs.logits).detach().cpu().numpy()  # Detach tensor before converting to NumPy
    
    # Create a DataFrame for the current prediction
    current_prediction = pd.DataFrame(preds, columns=label_columns[:-1])  # Exclude the "queue" column
    current_prediction["queue"] = example  # Add the "queue" column
    
    # Concatenate the current prediction to the predictions DataFrame
    predictions = pd.concat([predictions, current_prediction], axis=0, ignore_index=True)

# Reorder columns to have "queue" as the first column
predictions = predictions[["queue"] + label_columns[:-1]]

predictions

/tmp/ipykernel_27425/1235757157.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictions = pd.concat([predictions, current_prediction], axis=0, ignore_index=True)


,queue,is_immoral,intensity,불평/불만,환영/호의,감동/감탄,지긋지긋,고마움,슬픔,화남/분노,...,불안/걱정,기쁨,안심/신뢰,DISCRIMINATION,HATE,CENSURE,VIOLENCE,CRIME,SEXUAL,ABUSE
0,너희,0.101134,0.196528,0.376477,0.152148,0.107029,0.296697,0.043427,0.073531,0.258001,...,0.232043,0.060481,0.137384,0.024003,0.028506,0.123584,0.015065,0.015356,0.026285,0.017245
1,너희 집,0.081646,0.158600,0.266389,0.198563,0.128230,0.199411,0.057940,0.074929,0.155653,...,0.208553,0.114259,0.166078,0.025508,0.025466,0.080489,0.016953,0.015905,0.030830,0.016755
2,너희 집 개,0.654950,0.999990,0.308796,0.125560,0.150157,0.210892,0.025698,0.100603,0.260516,...,0.175678,0.097965,0.057927,0.063563,0.085728,0.513005,0.027069,0.019872,0.082977,0.028723
3,너희 집 개 귀엽다,0.167840,0.332623,0.090511,0.487590,0.556810,0.067602,0.147902,0.098262,0.069431,...,0.113365,0.544671,0.336895,0.047109,0.045260,0.159449,0.042371,0.053288,0.078950,0.055817
